In [1]:
import os
import pandas as pd
import numpy as np
import random
import lightgbm as lgb
import pickle

In [2]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'

In [3]:
df_train = pd.read_csv(FOLDER + 'train.csv')
df_test = pd.read_csv(FOLDER + 'test.csv')

In [ ]:
try_test = '20190611_test_dist_cos.csv'
trained_model = '20190614_champs_models_lgb_en.pickle'
submission_name = '20190615submission_2.csv'

In [ ]:
bond_types = df_train['type'].unique()
del df_train

In [ ]:
df_test_dist = pd.read_csv(OUTPUT + try_test)

In [ ]:
with open(OUTPUT + trained_model, 'rb') as f:
    models_0 = pickle.load(f)
    models_1 = pickle.load(f)
    models_2 = pickle.load(f)
    models_3 = pickle.load(f)
    models_4 = pickle.load(f)

In [ ]:
ids_arr = np.zeros(0)
pred_arr = np.zeros(0)
for i, b in enumerate(bond_types):
    df = df_test_dist.query('type == "{}"'.format(b))
    ids = df['id'].values
    ids_arr = np.hstack([ids_arr, ids])
    X = df[df.columns[5:]].values
    
    gbm_0 = models_0[i]
    gbm_1 = models_1[i]
    gbm_2 = models_2[i]
    gbm_3 = models_3[i]
    gbm_4 = models_4[i]
    
    y_pred_0 = gbm_0.predict(X)
    y_pred_1 = gbm_1.predict(X)
    y_pred_2 = gbm_2.predict(X)
    y_pred_3 = gbm_3.predict(X)
    y_pred_4 = gbm_4.predict(X)
    
    y_pred = (y_pred_0 + y_pred_1 + y_pred_2 + y_pred_3 + y_pred_4) / 5.
    
    pred_arr = np.hstack([pred_arr, y_pred])

In [ ]:
se_ids = pd.Series(ids_arr, name='id', dtype='int')
se_pred = pd.Series(pred_arr, name='scalar_coupling_constant')
df_submission = pd.concat([se_ids, se_pred], axis=1)
df_submission_s = df_submission.sort_values('id')
df_submission_s.to_csv(OUTPUT + submission_name, index=False)